In [1]:
!pip install selenium

In [2]:
!python --version

import selenium
print(selenium.__version__)

Python 3.12.9
4.29.0


In [58]:
!pip install mysql-connector-python -q

In [66]:
import mysql.connector
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup
import time

# MySQL 연결
args = {
    "host" : "localhost",       # MySQL 서버 주소
    "user" : "root",            # MySQL 사용자명
    "password" : "root1234",  # MySQL 비밀번호
    "database" : "car_project"  # 사용할 데이터베이스 이름
}
try:
    conn = mysql.connector.connect(**args)  # localhost 에 접속 해서 접속 객체를 생성
    cursor = conn.cursor()
    sql = """ 
    insert into car_project.brand_sales(brand,sales)
    """
    
    # 1. 다나와 자동차 판매 실적 url
    url = 'https://auto.danawa.com/auto/?Work=record&Tab=Grand&Month=2025-02-00'
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(1)
    
    # 2. 기간 선택 버튼 클릭
    radio_button = driver.find_element(By.XPATH,'//*[@id="monthPeriodDiv"]/span[2]/label/input')
    radio_button.click()
    
    # 3. 기간 from_year : 2025 -> 2024, from_month : 02 -> 01
    from_year = Select(driver.find_element(By.XPATH,'//*[@id="selMonthFrom"]'))
    from_year.select_by_value("2024")
    from_month = Select(driver.find_element(By.XPATH,'//*[@id="selDayFrom"]'))
    from_month.select_by_value("01")
    
    # 4. 기간 to_year : 2025 -> 2024, to_month : 03 -> 12
    to_year = Select(driver.find_element(By.XPATH,'//*[@id="selMonthTo"]'))
    to_year.select_by_value("2024")
    to_month = Select(driver.find_element(By.XPATH,'//*[@id="selDayTo"]'))
    to_month.select_by_value("12")
    
    # 5. 조회 버튼 클릭
    button = driver.find_element(By.XPATH,'//*[@id="monthPeriodDiv"]/span[2]/input')
    button.click()
    time.sleep(1)
    
    # 6. 페이지에서 HTML 추출 (BeautifulSoup 사용)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 7. 선긋기
    print('국내 자동차 브랜드 판매량 Top 6---------------------------')
    
    # 8. 국내 자동차 브랜드 순위가 포함된 테이블의 tr 태그들을 선택
    tr_lists = soup.select('#autodanawa_gridC > div.gridMain > article > main > div:nth-child(1) > table > tbody > tr')
    
    # 9. 국내 브랜드 및 판매량 정보 추출
    domestic_brand_sales = []
    for tr in tr_lists:
        Brand = tr.find('td', class_='title')  # 브랜드명
        Sales = tr.find('td', class_='num')  # 판매량
        
        if Brand and Sales:  # 데이터가 있는 경우
            Brand = Brand.text.strip()
            Sales = Sales.text.strip().replace(',', '')  # 쉼표 제거
    
            # 판매량을 숫자로 변환
            try:
                Sales = int(Sales)
            except ValueError:
                continue  # 판매량이 숫자로 변환되지 않는 경우 건너뜀
    
            # 국내 브랜드 정보 추가
            domestic_brand_sales.append([Brand, Sales])
    
    # 10. 국내 브랜드 순위 출력 (1위부터 6위까지)
    top_6_brands = domestic_brand_sales[:6]  # 상위 6개 브랜드만 추출
    for Brand, Sales in top_6_brands:
        print(f'Brand: {Brand}, Sales: {Sales}')
    
        # MySQL에 데이터 삽입
        cursor.execute("INSERT INTO brand_sales VALUES(null, %s, %s)", (Brand, Sales))
    conn.commit()   
    # 11. 선긋기
    print('해외 자동차 브랜드 판매량 Top 4---------------------------')
    
    # 12. 해외 자동차 브랜드 순위가 포함된 테이블의 tr 태그들을 선택
    tr_lists = soup.select('#autodanawa_gridC > div.gridMain > article > main > div:nth-child(2) > table > tbody > tr')
    
    # 13. 해외 브랜드 및 판매량 정보 추출
    Overseas_brand_sales = []
    for tr in tr_lists:
        Brand = tr.find('td', class_='title')  # 브랜드명
        Sales = tr.find('td', class_='num')  # 판매량
    
        if Brand and Sales:  # 데이터가 있는 경우
            Brand = Brand.text.strip()
            Sales = Sales.text.strip().replace(',', '')  # 쉼표 제거
    
            # 판매량을 숫자로 변환
            try :
                Sales = int(Sales)
            except ValueError :
                continue  # 판매량이 숫자로 변환되지 않는 경우 건너뜀
    
            # 해외 브랜드 정보 추가
            Overseas_brand_sales.append([Brand , Sales])
    
    # 14. 해외 브랜드 순위 출력 (1위부터 4위까지)
    top_4_brands = Overseas_brand_sales[:4]  # 상위 4개 브랜드만 추출
    for Brand , Sales in top_4_brands :
        print(f'Brand : {Brand}, Sales : {Sales}')
    
        # MySQL에 데이터 삽입
        cursor.execute("insert into brand_sales VALUES(null, %s, %s)", (Brand, Sales))
    
    conn.commit();
except Exception as e:
    print(e)
finally:
    if cursor: cursor.close()
    if conn: conn.close()


국내 자동차 브랜드 판매량 Top 6---------------------------
Brand: 현대, Sales: 558184
Brand: 기아, Sales: 540607
Brand: 제네시스, Sales: 130509
Brand: KGM, Sales: 47046
Brand: 르노코리아, Sales: 39816
Brand: 쉐보레, Sales: 24495
해외 자동차 브랜드 판매량 Top 4---------------------------
Brand : BMW, Sales : 73754
Brand : 벤츠, Sales : 66400
Brand : 테슬라, Sales : 29750
Brand : 볼보, Sales : 15051
